In [2]:
!pip install pulp

  Using cached PuLP-2.7.0-py3-none-any.whl (14.3 MB)


In [5]:
# Import necessary libraries and modules
import sys
sys.path.append('../src')

from data_loader import load_data
from data_preprocessing import preprocess_data
from optimization import optimize_fleet
from submission import generate_submission

import pandas as pd
import matplotlib.pyplot as plt

# Load and preprocess data
data = load_data()
data, master_data = preprocess_data(data)

# Display the first few rows of each dataset to understand their structure
for name, df in data.items():
    print(f"{name} dataset:\n", df.head(), "\n")

# Explore the demand dataset
print("Demand dataset summary:")
print(data['demand'].describe())
print("\n")

# Plot the carbon emissions limit over the years
carbon_emissions = data['carbon_emissions']
plt.figure(figsize=(10, 6))
plt.plot(carbon_emissions['Year'], carbon_emissions['Carbon emission CO2/kg'], marker='o')
plt.title('Yearly Carbon Emission Limits')
plt.xlabel('Year')
plt.ylabel('Carbon Emission CO2/kg')
plt.grid(True)
plt.show()

# Visualize fuel costs over the years
fuels = data['fuels']
for fuel_type in fuels['Fuel'].unique():
    fuel_data = fuels[fuels['Fuel'] == fuel_type]
    plt.plot(fuel_data['Year'], fuel_data['Cost ($/unit_fuel)'], label=fuel_type)

plt.title('Fuel Costs Over the Years')
plt.xlabel('Year')
plt.ylabel('Cost ($/unit_fuel)')
plt.legend()
plt.grid(True)
plt.show()

# Perform the optimization
results_df = optimize_fleet(data, master_data)

# Display the first few rows of the optimization results
print("Optimization Results:\n", results_df.head())

# Generate the submission file
generate_submission(results_df)

# Verify that the total distance covered meets the demand
total_demand = data['demand'].groupby(['Year', 'Size', 'Distance'])['Demand (km)'].sum()
total_covered = results_df[results_df['Type'] == 'Use'].groupby(['Year', 'Distance_bucket'])['Distance_per_vehicle(km)'].sum()

print("Total Demand vs Total Distance Covered:")
print(pd.concat([total_demand, total_covered], axis=1, keys=['Demand', 'Covered']))

# Verify carbon emissions constraint
total_carbon_emissions = results_df[results_df['Type'] == 'Use'].apply(
    lambda row: row['Distance_per_vehicle(km)'] * 
                data['vehicles_fuels'][data['vehicles_fuels']['ID'] == row['ID']]['Consumption (unit_fuel/km)'].values[0] * 
                data['fuels'][(data['fuels']['Fuel'] == row['Fuel']) & (data['fuels']['Year'] == row['Year'])]['Emissions (CO2/unit_fuel)'].values[0],
    axis=1
).groupby(results_df['Year']).sum()

print("Total Carbon Emissions vs Carbon Emission Limits:")
print(pd.concat([total_carbon_emissions, carbon_emissions.set_index('Year')['Carbon emission CO2/kg']], axis=1, keys=['Emissions', 'Limit']))


FileNotFoundError: [Errno 2] No such file or directory: 'data/carbon_emissions.csv'